In [ ]:
%pip install -qU langchain langchain_community langchain_core pymupdf4llm

In [193]:
import os
from langchain.schema import Document
import pymupdf4llm
from rich import print
from rich.progress import track
from typing import List

In [187]:
# RICH'S PRINT COLORS
YELLOW = "#fde047"
ORANGE = "#f97316"
RED = "#ef4444"
BLUE = "#3b82f6"
CYAN = "#06b6d4"
EMERALD = "#34d399"
VIOLET = "#a855f7"
PINK = "#ec4899"
GRAY = "#64748b"
WHITE = "#cccccc"

In [5]:
PDF_DIR = "../../../notificaciones"

In [189]:
def directory_loader(directory_path: str) -> List[Document]:
    """LOADS PDF DOCUMENTS FROM A GIVEN DIRECTORY WITH PROGRESS INDICATOR."""

    if not os.path.exists(directory_path):
        raise ValueError(
            f"directory_loader() >>> DIRECTORY {directory_path} DOESN'T EXIST."
        )

    loaded_docs: List[Document] = []

    # SEARCH IN THE GIVEN DIRECTORY FOR EACH PDF FILE IN IT AND GETS ITS PATH
    pdf_files_info = []
    for parent_dir_path, _, files in os.walk(directory_path):
        for filename in files:
            if filename.endswith(".pdf"):
                file_path = os.path.join(parent_dir_path, filename)
                pdf_files_info.append({"file_name": filename, "file_path": file_path})

    # LOADS EACH PDF FILE: FILE --> LIST[DOCUMENT]
    for file_info in track(pdf_files_info, description="LOADING FILES"):
        md_text = pymupdf4llm.to_markdown(file_info["file_path"], show_progress=False)
        md_text = md_text.replace("-----", "")
        md_text = md_text.replace("\n\n", "\n")
        md_text = md_text.replace("\n\n\n", "\n")
        md_text = md_text.strip()
        loaded_file = Document(metadata=file_info, page_content=md_text)

        loaded_docs.append(loaded_file)

    return loaded_docs

In [ ]:
docs = directory_loader(PDF_DIR)

In [191]:
len(docs)

6

In [ ]:
for index, doc in enumerate(docs):
    print(
        f"[bold {BLUE}]> DOC N°:[/] [bold {WHITE}]{index}[/]\n",
        f"[bold {EMERALD}]> FILENAME:[/] [bold {WHITE}]{doc.metadata["file_name"]}[/]\n\n",
        f"[bold {YELLOW}]> CONTENT:[/]\n[{WHITE}]{doc.page_content}[/]",
    )

---
---
---